<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise_13_1_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset with brain atlas A for classification tasks: for first set of 5 small programming tasks:
Oasis dataset provided directly by the nilearn package
Can be retrieved by downloading nilearn.datasets.fetch_oasis_vbm(n_subjects=100),
roughly 900 MB, structural brain scans (so-called voxel-based morphometry) for
classification of male and female individuals

Task 1- Use the PyMC3 package (https://docs.pymc.io) to implement a Bayesian
hierarchical Logistic Regression (tip: use PyMC3.Bernoulli() and PyMC3.invlogit()) to
classify sex. Each bottom-level region slope variable (zscored) should be modeled as a
Gaussian distribution with priors (mu=0, sigma=1). All subject data is fitted in the same quantitative
model, that is, all datapoints are used at once. As binary higher hierarchical level please
implement a hyperprior based on the variance component of the lower-level region slopes
by pooling through a higher-level HalfCauchy distribution for their respective lower-level
region slopes. As binary hyper-prior category please use left versus right brain side of the
100 schaefer-yeo atlas regions. 500 MCMC iterations is enough. Report output of
PyMC3.summary(credible_interval=0.90).

In [ ]:
%%capture
!pip install nilearn

In [ ]:
%%capture
import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
import nibabel as nib

NOTE: for this exercise I used PyMC version 5 instead of PyMC3 because the model ran better on the former (ie. less errors)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc as pm
import scipy.stats as stats
import arviz as az
from pymc import Bernoulli, Model, HalfCauchy
from pymc import invlogit, sample, summary
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_decomposition import PLSRegression as PLSR
from sklearn.cross_decomposition import CCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression as LR

In [ ]:
%%capture
brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
yeo = datasets.fetch_atlas_schaefer_2018(n_rois=100) # this needs to be modified for some tasks
masker = NiftiLabelsMasker(labels_img=yeo.maps, standardize=True, memory='nilearn_cache')
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = np.array(brain_data.ext_vars.mf == 'F', dtype=int) # gives 1 for females and 0 for males

In [ ]:
# We need to collect the indices of subjects with labels containing '_LH_' and '_RH_' separately
lh_indices = np.where([b'_LH_' in label for label in yeo.labels])[0]
rh_indices = np.where([b'_RH_' in label for label in yeo.labels])[0]
# Now let's calculate the length of these two vectors to find out how many ROIs there are in each hemisphere
print('There are a total of', len(lh_indices), 'left-hemisphere regions')
print('There are a total of', len(rh_indices), 'right-hemisphere regions')

There are a total of 50 left-hemisphere regions
There are a total of 50 right-hemisphere regions


In [ ]:
# Now that we know there are equal numbers of LH and RH regions, we're ready to define our Logistic Regression model
with pm.Model() as model:
    
    # We define a HalfCauchy hyperprior for the variance component of the lower-level region slopes (aka betas)
    hyperprior = pm.HalfCauchy('hyperprior', beta=1, shape=100)

    # We define a binary hyperprior category based on brain hemisphere (side) with left and right having equal probability (0.5/1)
    side = pm.Categorical('side', p=np.array([0.5, 0.5]))

    # We define the intercept term as a normal distribution with priors mu=0 and sigma=1
    intercept = pm.Normal('intercept', mu=0, sigma=1)
    
    # We define the bottom-level region slopes (or betas) as a normal distribution with priors mu=0, sigma=hyperprior
    beta_coefficients = pm.Normal('beta_coefficients', mu=0, sigma=hyperprior, shape=100)

    # We establish the desired linear model using input_variables as our "X"es and the required model parameters
    linear_model = intercept + input_variables @ beta_coefficients + side

    # We calculate the prior probabilities using invlogit function, as specified in the instructions
    probabilities = pm.invlogit(linear_model)

    # We define the likelihood as another Normal distribution with mean equal to probabilities, and sigma=1
    likelihood = pm.Normal('likelihood', mu=probabilities, sigma=1, observed=output_variable)

    # We sample from the posterior distribution to estimate the unseen variables using 500 MCMC draws.
    trace = pm.sample(500, return_inferencedata=True)
    # Let's define the summary table
    stats = pm.summary(trace, hdi_prob=0.9)

In [ ]:
# Finally, let's take a look at the summary table
from google.colab import data_table
data_table.DataTable(stats, include_index=True, num_rows_per_page=202)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
side,0.527,0.500,0.000,1.000,0.009,0.006,3000.0,1000.0,1.00
intercept,0.055,0.965,-1.579,1.549,0.046,0.037,437.0,463.0,1.01
beta_coefficients[0],-0.569,4.910,-4.053,3.260,0.566,0.486,164.0,129.0,1.01
beta_coefficients[1],-0.379,3.203,-4.261,2.266,0.197,0.156,251.0,164.0,1.00
beta_coefficients[2],-0.026,2.582,-3.087,4.770,0.244,0.173,131.0,125.0,1.02
...,...,...,...,...,...,...,...,...,...
hyperprior[95],3.221,10.156,0.009,6.945,0.828,0.587,120.0,183.0,1.02
hyperprior[96],1.796,3.911,0.008,3.797,0.239,0.169,166.0,230.0,1.00
hyperprior[97],2.897,7.642,0.007,4.943,0.604,0.428,87.0,224.0,1.04
hyperprior[98],2.892,5.797,0.008,7.025,0.540,0.383,67.0,145.0,1.03
